[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/joconnor-ml/osm-ai-tools/blob/master/notebooks/Classification.ipynb)

In [ ]:
#@title Authenticate, Import, Download Data

from google.colab import auth
auth.authenticate_user()

!pip install -q fsspec gcsfs

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import math

!mkdir data
!gsutil -m rsync -rd gs://osm-object-detector/data ./data
!mkdir pretrained_models
!gsutil -m rsync -rd gs://osm-object-detector/pretrained_models ./pretrained_models

In [ ]:
image_df = pd.read_csv("data/images.csv")
object_df = pd.read_csv("data/object_location_data_clustered.csv").merge(
    image_df, how="left", on="cluster_id", suffixes=("", "_image")
)
image_df

w = 1280
h = 1280
zoom = 17

def getPointLatLng(x, y, lat, lng):
    parallelMultiplier = math.cos(lat * math.pi / 180)
    degreesPerPixelX = 360 / math.pow(2, zoom + 8)
    degreesPerPixelY = 360 / math.pow(2, zoom + 8) * parallelMultiplier
    pointLat = lat - degreesPerPixelY * ( y - h / 2)
    pointLng = lng + degreesPerPixelX * ( x  - w / 2)

    return (pointLat, pointLng)

image_size = 0.01
def get_patch(row):
    ne = getPointLatLng(w, 0, row.center_lat_image, row.center_lon_image)
    sw = getPointLatLng(0, h, row.center_lat_image, row.center_lon_image)
    nw = getPointLatLng(0, 0, row.center_lat_image, row.center_lon_image)
    se = getPointLatLng(w, h, row.center_lat_image, row.center_lon_image)
    size_lat = ne[0] - se[0]
    size_lon = ne[1] - nw[1]
    return pd.Series(dict(
        y_min = (0.5 - (row.min_lat - row.center_lat_image)/size_lat)+0.06,  # add a small buffer
        y_max = (0.5 - (row.max_lat - row.center_lat_image)/size_lat)-0.06,
        x_min = (0.5 + (row.min_lon - row.center_lon_image)/size_lon)-0.06,
        x_max = (0.5 + (row.max_lon - row.center_lon_image)/size_lon)+0.06,
        osm_id = row.osm_id
    ))
patches = object_df.apply(get_patch, axis=1)
patches["image_id"] = object_df["image_id"]
patches["osm_id"] = patches["osm_id"].astype(int)

In [ ]:
patches.head()

In [ ]:
image_patches = []
patch_ids = []
for image_id in image_df.image_id:
    image_patches.append(patches.loc[patches["image_id"]==image_id, ["y_min", "x_min", "y_max", "x_max"]].values)
    patch_ids.append(patches.loc[patches["image_id"]==image_id, "osm_id"].values)

In [ ]:
def patch_gen():
    for coords in image_patches:
        yield coords
def patch_id_gen():
    for i in patch_ids:
        yield i
def label_gen():
    for p in image_patches:
        yield np.ones_like(p[:, 0], dtype=np.uint8) - 1

In [ ]:
# Load images and visualize
train_image_dir = 'data/images'
# Make a Dataset of file names including all the PNG images files in
# the relative image directory.
filename_dataset = tf.data.Dataset.from_tensor_slices(train_image_dir + "/" + image_df.image_id + ".png")
images = filename_dataset.map(lambda x: tf.io.decode_png(tf.io.read_file(x)))
bboxes = tf.data.Dataset.from_generator(patch_gen, output_types=tf.float32)
bbox_ids = tf.data.Dataset.from_generator(patch_id_gen, output_types=tf.int32)
images_and_bboxes = tf.data.Dataset.zip((images, bboxes, bbox_ids))
for img, bbox, bbox_id in images_and_bboxes.take(1):
    break
plt.imshow(img + 127)
bbox.numpy(), bbox_id.numpy()


In [ ]:
bboxes_per_image = object_df.shape[0] / image_df.shape[0]


In [ ]:
# generate positives -- grab crops for each bbox
IMAGE_SIZE = 224

def sample_positives(img, bboxes, bbox_ids):
    crops = tf.image.crop_and_resize(
        tf.expand_dims(img, axis=0), bboxes, box_indices=tf.zeros_like(bboxes[:, 0], dtype=tf.int32),
        crop_size=[IMAGE_SIZE, IMAGE_SIZE], method='bilinear',
        extrapolation_value=0, name=None
    )
    return tf.data.Dataset.zip((
        tf.data.Dataset.from_tensor_slices(crops),
        tf.data.Dataset.from_tensor_slices(bbox_ids),
        tf.data.Dataset.from_tensor_slices([1]).repeat(-1),
    ))

def sample_negatives(img, boxes, cls):
    return {"image": tf.cast(tf.image.random_crop(img, size=[IMAGE_SIZE, IMAGE_SIZE, 3]), np.float32), "bbox_id": -1, "label": 0}

positives = images_and_bboxes.flat_map(sample_positives).map(lambda img, box_id, cls: {"image": img, "bbox_id": box_id, "label": cls})
# use `repeat` to balance the data
negatives = images_and_bboxes.repeat(round(bboxes_per_image)).map(sample_negatives)
final_dataset = tf.data.experimental.sample_from_datasets([positives, negatives])

In [ ]:
for row in final_dataset.take(3):
    plt.imshow((row["image"].numpy() + 127).astype(np.uint8))
    plt.title(f"{row['bbox_id']}, {row['label']}")
    plt.show()


In [ ]:
# get size of dataset -- since we changed the number of rows dynamically we have to count them in full
for i, _ in enumerate(final_dataset.take(-1)):
    pass
num_samples = i+1
num_samples

In [ ]:
BATCH_SIZE = 128
half_the_data = int(num_samples/2)
train_ds = final_dataset.take(half_the_data)
val_ds = final_dataset.skip(half_the_data)

In [ ]:
def to_tuple(row):
    return row["image"], row["label"]

def get_model():
    module = tf.keras.models.load_model(os.path.join("pretrained_models", "resisc_224px_rgb_resnet50"))
    module.trainable = True
    module.summary()

    images = tf.keras.layers.Input((IMAGE_SIZE, IMAGE_SIZE, 3))
    data_augmentation = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
        tf.keras.layers.experimental.preprocessing.RandomContrast(0.1),
        tf.keras.layers.experimental.preprocessing.Resizing(256,256),
        tf.keras.layers.experimental.preprocessing.RandomTranslation(0.2, 0.2, fill_mode="constant"),
        tf.keras.layers.experimental.preprocessing.RandomRotation(2*math.pi, fill_mode="constant"),
        tf.keras.layers.experimental.preprocessing.RandomZoom(0.5, fill_mode="constant"),
        tf.keras.layers.experimental.preprocessing.CenterCrop(224,224),
    ])
    features = module(data_augmentation(images))
    features = tf.keras.layers.Concatenate(axis=-1)([
        tf.keras.layers.GlobalAveragePooling2D()(features),
        tf.keras.layers.GlobalMaxPooling2D()(features)
    ])
    features = tf.keras.layers.Dropout(0.5)(features)
    output = tf.keras.layers.Dense(1, activation="sigmoid")(features)
    model = tf.keras.Model(inputs=images, outputs=output)

    lr = 0.003 * BATCH_SIZE / 512

    # Decay learning rate by a factor of 10 at SCHEDULE_BOUNDARIES.
    lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(boundaries=[200, 300, 400], 
                                                                      values=[lr, lr*0.1, lr*0.001, lr*0.0001])
    optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9)

    model.compile(
      optimizer=optimizer,
      loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.01),
      metrics=['acc']
    )
    return model


In [ ]:
model = get_model()
model.fit(train_ds.map(to_tuple).shuffle(1000).batch(BATCH_SIZE).prefetch(2), validation_data=val_ds.map(to_tuple).batch(BATCH_SIZE).prefetch(2), epochs=10)

In [ ]:
pred_df = []
for row in val_ds.take(-1):
  if row["label"].numpy() == 0:
    continue
  pred = model.predict(tf.expand_dims(row["image"], axis=0)).item()
  pred_df.append({"pred": pred, "label": row["label"].numpy(), "osm_id": row["bbox_id"].numpy()})
pred_df = pd.DataFrame(pred_df)

In [ ]:
def plot_one_object(object_id):
    filename = patches.loc[patches.osm_id==object_id, "image_id"].iloc[0]
    img = tf.io.decode_png(tf.io.read_file(f"data/images/{filename}.png"))
    bboxes = patches.loc[patches.osm_id==object_id, ["y_min", "x_min", "y_max", "x_max"]].values
    crops = tf.image.crop_and_resize(
        tf.expand_dims(img, axis=0), bboxes, box_indices=tf.zeros_like(bboxes[:, 0], dtype=tf.int32),
        crop_size=[IMAGE_SIZE, IMAGE_SIZE], method='bilinear',
        extrapolation_value=0, name=None
    )
    plt.imshow((crops[0].numpy() + 127).astype(np.uint8))


In [ ]:
#@title Plot the 100 most surprising tagged cooling towers
for i, row in pred_df.nsmallest(100, "pred").iterrows():
  plot_one_object(row.osm_id)
  plt.title(f"{row.pred:.3f}, {row.label}, {row.osm_id}")
  plt.show()

I count a few strange square-shaped "cooling towers" -- possible these are correct labels, one clear correctly labelled tower and everything else is junk, which is great!

Note that some of the images of empty fields etc. may not be real mislabels, but newly constructed cooling towers for which our imagery is out of date. This should be confirmed by a human.

Either way, it looks like we have a fairly robust mislabel proposer. Let's finish the job.

In [ ]:
#@title Train on the second half of objects, predict on the first
model = get_model()
model.fit(val_ds.map(to_tuple).shuffle(1000).batch(BATCH_SIZE).prefetch(2), validation_data=train_ds.map(to_tuple).batch(BATCH_SIZE).prefetch(2), epochs=10)

In [ ]:
pred_df2 = []
for row in train_ds.take(-1):
  if row["label"].numpy() == 0:
    continue
  pred = model.predict(tf.expand_dims(row["image"], axis=0)).item()
  pred_df2.append({"pred": pred, "label": row["label"].numpy(), "osm_id": row["bbox_id"].numpy()})
pred_df2 = pd.DataFrame(pred_df2)

In [ ]:
pred_df2.nsmallest(50, "pred")

In [ ]:
#@title Plot the 100 most surprising tagged cooling towers
for i, row in pred_df2.nsmallest(100, "pred").iterrows():
  plot_one_object(row.osm_id)
  plt.title(f"{row.pred:.3f}, {row.label}, {row.osm_id}")
  plt.show()

In [ ]:
df = pd.concat([
  pred_df,
  pred_df2
])
df["mislabel_score"] = 1 - df["pred"]
df.to_csv("gs://osm-object-detector/mislabel_scores.csv")

In [ ]:
#@title Bonus: Rudimentary object detection
for img in images.take(10):
    images = []
    fig, ax = plt.subplots(1+(1280//IMAGE_SIZE),1+(1280//IMAGE_SIZE),figsize=(20,20), sharex=True, sharey=True)
    fig.tight_layout() # Or equivalently,  "plt.tight_layout()"

    for i, x in enumerate(range(0, 1260, IMAGE_SIZE)):
        for j, y in enumerate(range(0, 1260, IMAGE_SIZE)):
            image = np.zeros((IMAGE_SIZE, IMAGE_SIZE, 3))
            new_image = img.numpy()[y:y+IMAGE_SIZE, x:x+IMAGE_SIZE, :]
            image[:new_image.shape[0], :new_image.shape[1], :] = new_image
            pred = model.predict(tf.expand_dims(image, axis=0))[0][0]
            if pred < 0.25:
              image *= 0.25  # darken panels with no detections for emphasis
            else:
              image *= pred
            ax[j][i].imshow(image.astype(np.uint8))
            ax[j][i].set_title(pred)
    plt.show()